# Ghigliottina

## Ipotesi fatte

Nell'esecuzione di questo esercizio sono state fatte una serie di ipotesi in modo da semplificarne la risoluzione. Tali ipotesi sono qui elencate:
1) La lingua scelta è l'inglese.
2) Ammettiamo che i legami tra le cinque parole date siano unicamente dati da modi di dire.
3) Il sistema deve sempre restituire una parola in output. Se abbiamo due o più parole che potrebbero essere la parola in output con la medesima probabilità, ne scegliamo una casualmente. 
4) Come nel gioco originale, consideriamo che il singolare e il plurale di una parola siano due termini diversi.

### Terminologia

Per chiarezza esplicativa, in questo notebook useremo:
- __parola__: per indicare una delle cinque parole date in input al sistema, oppure la parola di output
- __termini__: tutti i termini che vengono utilizzati durante la computazione per trovare quale tra essi quello più adatto ad essere la parola di output
- __(nel codice) quote__: un modo di dire

## Import

In [33]:
from random import randint
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

## Risorse utilizzate

Una conseguenza dell'ipotesi (2) è che per risolvere questo esercizio è sufficiente avere una raccolta di modi di dire, il file `common_saying.txt`. Essa è stata costruita a mano, prendendo circa 2500 modi di dire in inglese trovati nel Web e sistemandone uno per riga. Ciascuno di questi modi di dire non è stato sottoposto ad alcuna operazione di pulizia, e quindi potrebbe presentare caratteri quali le parentesi o il trattino.

Un'altra risorsa che viene utilizzata è il file `words.txt`. Questo contiene, in ogni riga, una serie di cinque parole, che sono i diversi possibili input al sistema.

#### Path delle risorse

In [34]:
QUOTES_PATH = 'resources/common_saying.txt'
WORDS_PATH = 'resources/words.txt'

### Funzioni di manipolazione dei dati

Semplice funzione che permette di rimuovere stopword e punteggiatura da un termine. Non utilizziamo la lemmatizzazione in quanto, per l'Ipotesi (4) vogliamo fare distizione tra singolari e plurali.

In [35]:
def preprocess_term(term):
    delete_punctuation_tokenizer = RegexpTokenizer(r'\w+')
    term = term.lower()
    if not term in stopwords.words():
        no_punct_term = delete_punctuation_tokenizer.tokenize(term)
        if len(no_punct_term) > 0:
            return no_punct_term[0]
    return None

### Manipolazione dei modi di dire

Viste le premesse, è necessario avere delle funzioni ad hoc che permettano una comoda manipolazione dei modi di dire. In particolare ci servirà una funzione per leggere i modi di dire dal file, e un'altra che permetta di togliere i caratteri speciali laddove serva. In ultimo una funzione che sia in grado di estrarre i termini significativi da una frase.

In [36]:
def get_quotes(path):
    quotes = []
    with open(path, 'r', encoding='utf-8') as f:
        quotes = f.readlines()
    return quotes
    
def preprocess_quote(quote):
    if '(' in quote:
        quote = quote.replace('(', '').replace(')', '')
    if '-' in quote:
        for _ in range(quote.count('-')):
            p = quote.find('-')
            quote = quote[:p] + ' ' + quote[p+1:]
    return quote.lower().strip('\n')

def extract_terms(quote):
    terms = list()
    for term in quote.split(' '):
        term = preprocess_term(term)
        if term is not None:
            terms.append(term)
    return terms

### Estrazione delle cinque parole

In [37]:
def select_words(path):
    words = []
    with open(path, 'r') as f:
        lines = f.readlines()
        words = lines[randint(0,len(lines)-1)].strip('\n').split(' ')
    return words

## Risoluzione

La raccolta di modi di dire viene utilizzata per ricercare le cinque parole date, in modo da ottenere, per ciascuna parola, un insieme di frasi ad essa legate. I termini contenute in tali frasi vengono quindi utilizzati come insieme di contesto per ciascuna delle cinque parole date.

In [38]:
def build_contexts(words, path):
    quotes = get_quotes(path)
    res = {}
    for word in words:
        res[word] = set()

    for quote in quotes:
        quote = preprocess_quote(quote)
        for word in words:
            if word in quote.split(' '): 
                terms = extract_terms(quote)
                res[word].update(terms)
    return res

A questo punto è necessario contare le occorrenze nei cinque diversi insiemi di contesto per ciascuno dei termini. Se uno di essi dovesse essere uguale a una delle parole, è da escludere dai possibili risultati.

In [39]:
def count_term_occurrences(context):
    res = dict()
    for word in context:
        for term in context[word]:
            if term in context:
                continue
            if term not in res:
                res[term] = 0
            res[term] += 1
    return res

La parola data in output dal sistema sarà il termine (o uno dei termini) che compare nel maggior numero di insiemi di contesto diversi. Il sistema dà in output anche il grado di confidenza, misurato come il numero di occorrenze del termine diviso 5.

In [40]:
def execute(quotes_path, words_path):
    words = select_words(words_path)
    print("Five words where selected: {}".format(words))
    contexts = build_contexts(words, quotes_path)
    terms = count_term_occurrences(contexts)
    sorted_terms = sorted(terms.items(), key=lambda x: x[1], reverse=True)
    for term in sorted_terms:
        if term not in words:
            print("The most probable word linked to the given five is: {}, with confidence: {}".format(term[0], term[1]/5))
            break

execute(QUOTES_PATH, WORDS_PATH)

Five words where selected: ['tape', 'sky', 'letter', 'see', 'rag']
The most probable word linked to the given five is: red, with confidence: 1.0


## Limiti del sistema

### Mancanza del Senso Comune

Un forte limite del sistema è la mancata cattura di legami semantici dati dal senso comune, che è una diretta conseguenza dell'Ipotesi (2). Un modo per risolvere tale problema sarebbe quindi rilassarla. Per farlo si potrebbe migliorare la costruzione dell'insieme di contesto sfruttando risorse come ConceptNet per catturare anche il senso comune. In particolare, si potrebbero aggiungere a suddetto insieme di contesto, i nodi vicini (entro un certo raggio) alla parola cercata. 
Un esempio di questo limite è dato dalle cinque parole _friday, tires, travel, road, crash_. Qui un umano individua facilmente che la risposta è la parola _car_. Tuttavia non essendoci modi di dire che leghino le ultime 4 parole a _car_, il sistema attualmente non è in grado di trovare con sicurezza una sola parola. Per l'Ipotesi (3), esso restituisce comunque una parola, e può capitare che essa sia proprio _car_, ma solo perché viene selezionata casualmente.